In [1]:
import os
import sys
import json

import pandas as pd

os.chdir(os.path.join("..", ".."))

from solution.constants import REVIEWS_PATH, PRODUCTS_PATH, RETURN_REASONS_PATH, RETURNS_PATH, TEST_PATH

In [2]:
from solution.fasttext import embed_sentences

In [3]:
import fasttext

model_path = os.path.join("models", "lid.176.bin")
model = fasttext.load_model(model_path)

# Text for language identification
text = "Вас с морем соли"
text = "bu ajoyib Sifat zòr Raxmat"


# Predict the language
predicted_language = model.predict(text, k=1)  # k=1 means only top 1 prediction will be returned
language_code = predicted_language[0][0].split('__label__')[1]

print("Predicted Language Code:", language_code)


Predicted Language Code: pam


In [4]:
class LanguagePredictor:
    def __init__(self, model_path: str):
        self.model = fasttext.load_model(model_path)

    def predict_language(text: str):
        predicted_language = model.predict(text, k=1)
        language_code = predicted_language[0][0].split('__label__')[1]

        return language_code

In [5]:
import fasttext

# Load pre-trained FastText model
model_path = 'fasttext\cc.uz.300.bin'  # Replace with the path to your pre-trained model
model = fasttext.load_model(model_path)

# Function to tokenize Uzbek text using FastText
def tokenize_uzbek(text):
    tokens = model.get_sentence_vector(text)
    return tokens

# Example Uzbek text
uzbek_text = "Ҳаётнинг кутубхоналардаги оқувчиларга олмаган мустақил олимпиада"

# Tokenize Uzbek text
uzbek_tokens = tokenize_uzbek(uzbek_text)
print(uzbek_tokens)


[-1.04140593e-02 -1.34027805e-02  5.66064529e-02  4.95695472e-02
  4.90198694e-02 -6.34529889e-02  5.05424365e-02 -2.06366982e-02
  3.26077342e-02 -1.08819520e-02 -1.12000294e-03  6.28365576e-02
 -5.11389691e-03 -6.49911761e-02 -4.35957685e-02  4.77548838e-02
 -2.35173758e-02  4.87036519e-02  2.98887622e-02  4.27192301e-02
 -2.54738773e-03  2.04317700e-02 -1.50741097e-02 -4.48082797e-02
 -2.08223928e-02  2.34554913e-02 -6.84142411e-02  6.39473349e-02
  4.92517352e-02 -3.29608321e-02 -1.78260542e-02 -3.75965461e-02
  3.56050655e-02 -1.77604926e-03  9.87625122e-02  2.43951436e-02
 -5.87348342e-02 -3.56631260e-03  2.08634287e-02 -1.15864724e-02
  1.50687769e-02 -3.29675451e-02 -2.50840690e-02 -1.53332874e-02
 -1.41639579e-02 -6.56649657e-03 -3.03235091e-02 -1.39161227e-02
 -4.01627868e-02 -5.59557192e-02  4.94836047e-02  1.41262114e-02
  9.22106206e-02  6.56379759e-02 -6.76241666e-02  2.05602534e-02
 -1.84132215e-02  5.15088700e-02  2.57402211e-02  4.66423556e-02
  3.28033976e-02  1.56532

In [6]:
len(uzbek_tokens)

300

# Imports

# Embdeddings

## Review

In [7]:
reviews_df = pd.read_parquet(REVIEWS_PATH)
reviews_df.head()

,order_item_id,product_id,customer_id,review_text,shop_id,rating,date_created
0,19068657,641695ab1f06465947d1e4e26f61ed308145dce718d9b6...,0aaefdb4bd01799d678f7fcd453e23c597804eb006f1ce...,товар потерян,c2f61742e5f0ef0a74c5fcbeff7ff659def91bf4b5f1e1...,1,2023-10-17 17:26:42.874
1,11130626,2c0818e03896a3e27fd8ec01ba9d7ae80d97673575a847...,851513a761b89239303515254d0e9c22a8474dd2df60b8...,,757db91a80964d58a2b0d26bffd641bcbe142aae4f7f47...,5,2023-07-27 10:06:40.711
2,8992153,d00a0a981e5618c4f96802e79c205ef2175b141edff368...,96659e77ab5e9b52ecbf110eed661c3fe742fc42443d50...,,8f0f97e140e126a4404a09eb1e14a53b7c742701c4290a...,5,2023-07-03 14:47:59.316
3,11078898,d0d9bad11fa6fedcf5c699055137a36f4c4c81d722b5fd...,321667e5d0ada2fa7f2356076f88a3848e596cde7bd120...,"Rahmat, juda yoqdi!",61182f39851829ca78c919a83ecbfa045fc0686bff16d0...,5,2023-07-30 12:26:59.864
4,32080499,4edd99b2e2a203a18cf24b6a2b134833ef1f74cf1940c0...,d67c451b69c7f706cedf6d4a87f1180ea62acd57c94a29...,16гб работает хорошо,02cca3803b564ede11ccf9f303c9910b39c532061e7a8c...,5,2024-01-19 06:02:57.971


In [23]:
reviews_df[reviews_df.rating < 4].reset_index(drop=True).groupby("product_id").indices

{'0000965e3e7fc300989c81f6e50c9d224d09f93434925af38d2f2f78e5aabc0b': array([ 93161, 100568, 103380, 106635, 110423, 111520, 116114],
       dtype=int64),
 '0000a928e398e3349f3aedc84c3eb0a5988fdba3a2c07a6a53ec6154ff92d8e5': array([ 24543,  28137,  35014,  40138,  45114,  48100,  48441,  52055,
         67649,  73382, 102220, 105378], dtype=int64),
 '0000ccb6fbc0156d34fa585c68626ca1526dcef175034761145ab0ca242edf11': array([107913, 107914, 150103, 153160, 181639, 182337, 182553],
       dtype=int64),
 '000470eaa5d7414df9dc2f30a552bf78f5daf426aabe43b180b3c4c839d5b2bc': array([14240], dtype=int64),
 '00073af95d61805cdf2036768534da7147e075c801c25aeec971ef91a4942025': array([113988, 152752], dtype=int64),
 '000a1df64c4814b25180e09a3626b6ed0f013179b412f65510a73c3540dd5d0a': array([17444], dtype=int64),
 '000cda0704a843ff7cb7137cc2a87bddfa7390ea5561c3cc51ac0ff05eb8ad98': array([ 16282,  16725, 138859, 158792], dtype=int64),
 '0011ac276e06b1ca66e40714f9bc33eeb5690afdecf776d631687ba5eff263a3': ar

In [8]:
sum(reviews_df[reviews_df.rating < 4].review_text.isnull())

0

In [9]:
emb = embed_sentences(reviews_df[reviews_df.rating < 4].review_text[:10])

100%|██████████| 10/10 [00:00<00:00, 9306.20it/s]


In [10]:
reviews_emb = embed_sentences(reviews_df[reviews_df.rating < 4].review_text)

100%|██████████| 183726/183726 [00:09<00:00, 19535.49it/s]


In [14]:
reviews_emb.shape

(183726, 300)

In [11]:
emb.shape

(10, 300)

In [12]:
emb

array([[ 0.11942215, -0.01334448,  0.00983252, ...,  0.03422549,
        -0.05172918,  0.00202305],
       [ 0.02527054,  0.00056255,  0.00761171, ...,  0.01712064,
        -0.04036796, -0.03787952],
       [-0.02076377, -0.0011848 ,  0.02202   , ..., -0.04929919,
         0.01339135,  0.05504996],
       ...,
       [-0.01011243, -0.01206531, -0.0120538 , ...,  0.0159256 ,
         0.00337603, -0.04846438],
       [-0.01532095, -0.0403511 ,  0.0094301 , ..., -0.01147936,
         0.03665285,  0.01124927],
       [ 0.02792926, -0.00899615, -0.00781589, ...,  0.01622477,
        -0.01032414, -0.01825982]], dtype=float32)

In [13]:
emb[0]

array([ 0.11942215, -0.01334448,  0.00983252,  0.03307826,  0.00983474,
       -0.00020968,  0.04808807,  0.01313598, -0.00471829,  0.06413313,
        0.01854319, -0.00978077,  0.05086241,  0.02943244, -0.03881898,
        0.0144665 ,  0.07550614, -0.03588115, -0.0054996 ,  0.0468456 ,
        0.05386879, -0.050842  ,  0.00188307,  0.01362205,  0.03216771,
        0.00852447, -0.03090175,  0.0068125 ,  0.05731192,  0.05779202,
        0.01648194, -0.02713797, -0.07969891, -0.04928545, -0.02032172,
       -0.01736476, -0.032605  , -0.10694912,  0.03797648,  0.05236403,
        0.07518313, -0.045727  , -0.03907427,  0.02774167,  0.04885841,
       -0.01694364, -0.03265455, -0.00850579,  0.13119747,  0.0009381 ,
        0.02783358, -0.02816302, -0.00983279,  0.01841585,  0.04770828,
        0.06211233,  0.02318943, -0.02412565,  0.01148804, -0.01135338,
       -0.07980122,  0.01448862,  0.01637313,  0.00066754, -0.02871859,
        0.00663429, -0.00177462, -0.0061102 , -0.01993926, -0.03